### to select correct number of hidden layers

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix,accuracy_score

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense
from scikeras.wrappers import KerasClassifier

In [4]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:,3:13]
y = dataset.iloc[:,13]




In [5]:
categorical_features = ['Geography','Gender']
ct = ColumnTransformer(
    transformers=[('encoder',OneHotEncoder(drop = 'first'),categorical_features)],
    remainder="passthrough"
)
X = ct.fit_transform(X)

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [7]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
from tensorflow.keras.layers import Dense,Activation
from tensorflow.keras.models import Sequential

def create_model(activation="relu", neurons=32):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train.shape[1], activation=activation))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model




In [9]:
# Wrap with scikeras
clf = KerasClassifier(model=create_model, verbose=0)


In [10]:
# Define grid
param_grid = {
    "model__activation": ["relu", "tanh"],
    "model__neurons": [16, 32, 64],
    "batch_size": [16, 32],
    "epochs": [10, 20]
}

In [11]:
# Run GridSearchCV
grid = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3, n_jobs=1)
grid_result = grid.fit(X_train, y_train)


c:\AI_ML_fresh\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\AI_ML_fresh\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\AI_ML_fresh\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\AI_ML_f

In [ ]:
print("Best Score:", grid_result.best_score_)
print("Best Params:", grid_result.best_params_)


Best Score: 0.8579993704837907
Best Params: {'batch_size': 32, 'epochs': 20, 'model__activation': 'tanh', 'model__neurons': 64}


In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
y_pred = grid.predict(X_test)
y_pred = (y_pred >0.5)

In [15]:
cm = confusion_matrix(y_test,y_pred)
acc = accuracy_score(y_test,y_pred)
print(cm)
print(acc)

[[1527   68]
 [ 203  202]]
0.8645
